In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0, InceptionV3, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report

dataset_dir = "/Users/Academics/Sem6/Major Project/Cotton Data Set/SAR-CLD-2024 A Comprehensive Dataset for Cotton Leaf Disease Detection/Original Dataset"  # Update this path to your dataset directory
categories = ["bacteria blight", "curl Virus", "healthy leaves", "herbicide growth damage", 
              "leafe hopper jassids", "leafe Redding", "leafe variegation", "invalid"]  # Add "invalid" class


img_width, img_height = 224, 224
batch_size = 32
num_classes = len(categories)

datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Function to build a transfer learning model
def build_model(base_model, num_classes):
    base_model.trainable = False  # Freeze the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)  # Add dropout for regularization
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# List of pre-trained models to compare
models = {
    "MobileNetV2": MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3)),
    "ResNet50": ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3)),
    "EfficientNetB0": EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3)),
    "InceptionV3": InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3)),
    "DenseNet121": DenseNet121(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
}


results = {}
for model_name, base_model in models.items():
    print(f"Training {model_name}...")
    
    
    model = build_model(base_model, num_classes)
    
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    
    checkpoint = ModelCheckpoint(
        f"{model_name}_best_model.h5",
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )
    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    )
    
    
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        epochs=20,
        callbacks=[checkpoint, early_stopping]
    )
    
   
    val_loss, val_accuracy = model.evaluate(validation_generator)
    results[model_name] = val_accuracy
    print(f"{model_name} Validation Accuracy: {val_accuracy * 100:.2f}%")


print("\nModel Comparison:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy * 100:.2f}%")


best_model_name = max(results, key=results.get)
best_accuracy = results[best_model_name]
print(f"\nBest Model: {best_model_name} with Accuracy: {best_accuracy * 100:.2f}%")

best_model = tf.keras.models.load_model(f"{best_model_name}_best_model.h5")
best_model.save("best_cotton_disease_model.h5")
print("Best model saved as 'best_cotton_disease_model.h5'.")

def predict_image(image_path, model, threshold=0.9):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)
    
    if confidence < threshold:
        return "Invalid image (not a cotton leaf)"
    else:
        return categories[predicted_class]


random_image_path = "path_to_random_image.jpg"  
result = predict_image(random_image_path, best_model)
print(f"Prediction for random image: {result}")


Found 1711 images belonging to 8 classes.
Found 426 images belonging to 8 classes.


Training MobileNetV2...
Epoch 1/20


2025-03-03 00:51:44.357929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - ETA: 0s - loss: 1.5773 - accuracy: 0.4568

2025-03-03 00:52:07.458659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/AgriGo3/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

2025-03-03 01:12:26.970586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - ETA: 0s - loss: 2.0083 - accuracy: 0.3210

2025-03-03 01:13:05.357333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - 51s 891ms/step - loss: 2.0083 - accuracy: 0.3210 - val_loss: 1.3514 - val_accuracy: 0.5409
Epoch 2/20
53/53 [==============================] - 48s 910ms/step - loss: 1.1506 - accuracy: 0.5849 - val_loss: 1.0827 - val_accuracy: 0.5793
Epoch 3/20
53/53 [==============================] - 44s 825ms/step - loss: 0.8869 - accuracy: 0.6790 - val_loss: 0.9127 - val_accuracy: 0.6779
Epoch 4/20
53/53 [==============================] - 48s 911ms/step - loss: 0.7376 - accuracy: 0.7367 - val_loss: 0.8510 - val_accuracy: 0.6827
Epoch 5/20
53/53 [==============================] - 44s 833ms/step - loss: 0.6188 - accuracy: 0.7880 - val_loss: 0.8270 - val_accuracy: 0.6803
Epoch 6/20
53/53 [==============================] - 45s 841ms/step - loss: 0.5880 - accuracy: 0.7820 - val_loss: 0.8170 - val_accuracy: 0.7091
Epoch 7/20
53/53 [==============================] - 45s 860ms/step - loss: 0.5155 - accuracy: 0.8148 - val_loss: 0.7570 - val_accuracy: 0.7139
Epoch 8/20

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_random_image.jpg'

In [ ]:
random_image_path = "/Users/Academics/Sem6/Major Project/Cotton Data Set/SAR-CLD-2024 A Comprehensive Dataset for Cotton Leaf Disease Detection/Original Dataset/Curl Virus/CV00001.jpg" 
result = predict_image(random_image_path, best_model)
print(f"Prediction for random image: {result}")

1/1 [==============================] - 0s 75ms/step
Prediction for random image: Invalid image (not a cotton leaf)
